# Lesson 3: The Verifier (Best-of-N)

In Lesson 2 (which we skipped to get here), we saw that **Majority Voting failed** for the math problem "Is 91 prime?".
The model `mistral:7b` confidently said "Yes" 5 times out of 5.

This happens when a model has a strong **System 1 bias** (intuition) that is incorrect.
To fix this, we need **System 2 thinking** (Reasoning) and a **Verifier**.

**New Strategy:**
1.  **Generate** reasoning traces ("Think step by step...").
2.  **Verify** the logic (Is the math correct?).
3.  **Select** the best answer based on verification scores (Best-of-N).

In [1]:
import time
from rich.console import Console
try:
    import ollama
except ImportError:
    print("pip install ollama")

console = Console()
MODEL_NAME = "mistral:7b" 

## 1. The Generator (Thinker)
We ask the model to "Think step-by-step". This encourages it to output a Chain of Thought (CoT).
Often, the mere act of writing out the steps allows the model to catch its own error.

In [2]:
def generate_thought(prompt: str, temp: float = 0.7) -> str:
    """Generates a step-by-step reasoning trace."""
    response = ollama.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are a math expert. Think step-by-step to check for factors. show your work."},
            {"role": "user", "content": prompt}
        ],
        options={"temperature": temp}
    )
    return response['message']['content'].strip()

## 2. The Verifier (LLM-as-a-Judge)
In a production TTRL system, this would be a trained Reward Model.
Here, we act as a "Self-Correction" system by asking the model to look at its own work and judge it.
We ask: *"Check the math. Determine if the reasoning is sound."*

In [3]:
def verify_solution(problem: str, solution: str) -> float:
    """
    Asks the model to critique the reasoning. Returns a score 0.0 to 1.0.
    """
    verifier_prompt = f"""
    Problem: {problem}
    Proposed Solution: {solution}
    
    Task: Check the math calculations in the solution. 
    If you find ANY calculation error (e.g. 7*13 != 91), score it 0.
    If the reasoning is sound and concludes correctly, score it 1.
    Reply with ONLY the score (0 or 1).
    """
    
    response = ollama.chat(
        model=MODEL_NAME, # Self-Correction
        messages=[{"role": "user", "content": verifier_prompt}],
        options={"temperature": 0.0} # Deterministic for judging
    )
    
    content = response['message']['content'].strip()
    return 1.0 if "1" in content else 0.0

## 3. Best-of-N Search Loop
We run the loop:
1.  Generate a solution.
2.  Verify it.
3.  Keep the one with the highest Verification Score.

This is simpler than PPO training but often yields massive gains at inference time (like **OpenAI o1**).

In [4]:
PROBLEM = "Is 91 a prime number?"
N_SAMPLES = 5

console.print(f"\n[bold yellow]Running Best-of-N Search (N={N_SAMPLES})...[/bold yellow]")
console.print(f"Problem: {PROBLEM}")

best_score = -1.0
best_solution = ""

for i in range(N_SAMPLES):
    # 1. Generate
    solution = generate_thought(PROBLEM, temp=0.8)
    
    # 2. Verify
    score = verify_solution(PROBLEM, solution)
    
    color = "green" if score > 0.5 else "red"
    console.print(f"\n[bold]Sample {i+1} (Score: {score}):[/bold]")
    console.print(f"[{color}]{solution[:150].replace('\n', ' ')}...[/{color}]")
    
    if score > best_score:
        best_score = score
        best_solution = solution

Running Best-of-N Search (N=5)...

Problem: Is 91 a prime number?

Sample 1 (Score: 1.0):

To determine if a number is prime, we check if it has any factors other than 1 and itself. Here's the step-by-step 
process for checking if 91 is a pri...

Sample 2 (Score: 1.0):

...

To determine if 91 is a prime number, we will check if it has any divisors other than 1 and itself. Since 1 is not 
considered a proper divisor, let's ...

Sample 4 (Score: 1.0):

To determine if a number is prime, we check if it has any divisors other than 1 and itself. Let's see if 91 has any
factors other than 1 and 91:  1. C...

Sample 5 (Score: 1.0):

To determine if a number is prime, we need to check if it has any factors other than 1 and itself. For the number 
91, let's list its possible factors:...

## 4. Final Result
If TTRL works, we should find at least one solution that realized 91 is divisible by 7.

In [5]:
console.print(f"\n[bold cyan]🏆 Best Verification Score:[/bold cyan] {best_score}")

if best_score == 1.0:
    console.print("[bold green]TTRL SUCCESS:[/bold green] Found a valid reasoning path!")
    console.print(f"Best Thought:\n{best_solution}")
else:
    console.print("[bold red]FAILURE:[/bold red] Could not find a verified solution.")

🏆 Best Verification Score: 1.0

TTRL SUCCESS: Found a valid reasoning path!

Best Thought:
To determine if a number is prime, we check if it has any factors other than 1 and itself. Here's the step-by-step 
process for checking if 91 is a prime number:

1. First, let's list the first few multiples of 91:
   Multiples of 91 are: 91, 182, 273, 364, 455, ... (and so on)

2. Notice that the smallest factor of 91 is actually 1 and itself (91). But we're looking for factors larger than 1
and less than 91.

3. We try dividing 91 by the numbers from 2 to its square root, which is approximately 9.545:
   - 2 * 91 = 182 (already listed as a multiple)
   - 3 * 30.333... = 90.999... (this is not a whole number, so we skip it)
   - 4 * 22.75... = 90.75... (same here, not a whole number)
   - ...

4. Since we didn't find any factors of 91 in the range from 2 to its square root, we can conclude that:

91 is a prime number because it only has two distinct positive divisors: 1 and itself (91).